# Function Dependency Equation for Free Product of 2 Cyclic Groups.

Consider $G=\langle x| x^m=1 \rangle * \langle y| y^n=1 \rangle$, where $m,n \in \mathbb{Z}_{\ge0}\setminus \{1\}$. Note that $|G|<\infty \iff mn>0$. 
Take the generating set $S=S_x\cup S_y$ for $G$, where 
$$S_x=\{x\}\cup \iota(m=0)\cdot\{x^{-1}\}, S_y=\{y\}\cup \iota(n=0)\cdot\{y^{-1}\},$$ 
and $$\iota(\mathcal{P})=\begin{cases}\{1\},&\mathcal{P}\\ \emptyset,& \lnot \mathcal{P}\end{cases}$$ with $\cdot$ denoting elementwise group multiplication on sets [$A\cdot B = \{ab: a\in A,\ b\in B\}$]. 

For $g\in G$ and $X\subseteq G$, let $F_{g,X}(t)$ be series for the set of all words in $S^*$ equivalent to $g\in G$, with proper nonempty prefixes avoiding $X$, characterized by the word length. We set up functional equations involving $F_{g,X}$, for which $\{g\}\cup X\subseteq \{v^k: k\in \mathbb{Z}\}$, and $v\in \{x,y\}$. Finally, we solve for $F(t):=F_{1,\emptyset}(t)$.

In [241]:
# choose your desired values for m,n

m = 3
n = 0

# Do we want word concatenation to reduce to group multiplication?
AUTO_REDUCE = True
DEFAULT_RS = None # a rewiting system in order to make group element reduction possible, will be set later


In [242]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    if AUTO_REDUCE and DEFAULT_RS is not None:
        C = Set([DEFAULT_RS.reduce(c) for c in C])
    return C

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [243]:
# compute the free product
F2.<x,y> = FreeGroup()
G = F2/ (x^m,y^n)
x,y = G.gens()

# need to write words in reduced form
Grs = G.rewriting_system()
DEFAULT_RS = Grs

In [244]:
# # test it out
# A = Set([x,y,G.one()])

# print(x*A) # with reduction

# # very simple to supress/invoke group reduction
# AUTO_REDUCE = False
# print(x*A) # without reduction
# AUTO_REDUCE = True
# print(x*A) # with reduction



In [245]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\emptyset'
    return latex(A)

In [246]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

add_series_var(goal_pair)
    

True

In [247]:
pretty_print(var_space)

{(1, {}): v0}

In [248]:
# Now get the cyclic factors

# Sage is not good with subgroups
# Gx = G.subgroup((x,))
# Gy = G.subgroup((y,))
# xx,yy = Gx.gens()+Gy.gens()

def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: either x or y, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume finite sets only
    return all((in_cyclic_factor(el,v) for el in A))

In [249]:
# for finite cyclic factors, we don't want negative exponents. We redefine the __pow__ method.
if AUTO_REDUCE and DEFAULT_RS is not None:
    if '_old_pow' not in vars():
        _old_pow = type(x).__pow__
    def _new_pow(a, k):
        if k not in ZZ or k>=0:
            # case isn't broken, don't change it
            return _old_pow(a,k)

        # k is a negative integer
        ans = _old_pow(a,k)
    #     if AUTO_REDUCE:
    #         ans = DEFAULT_RS.reduce(ans)
        nans = G.one()
        for v,ep in ans.syllables():
            modval = m if v==x else n # charateristic of the free factor
            nans*= _old_pow(v, ep if modval==0 else ep%modval) # if infinite oder, keep same exponent
        return nans
    type(x).__pow__ = _new_pow

In [250]:
# For multiplication, want auto reduation to be available
# if AUTO_REDUCE and DEFAULT_RS is not None:
#     if '_old_mul' not in vars():
#         _old_mul = type(x).__mul__
#     def _new_mul(a, b):
#         return DEFAULT_RS.reduce(_old_mul(a,b))
#     type(x).__mul__ = _new_mul


# set up the generating sets
Sx = Set([x]+[x^-1]*int(m==0))
Sy = Set([y]+[y^-1]*int(n==0))
Sx,Sy

({x}, {y, y^-1})

In [251]:
x,x^2,x^3*x,x^-1,(x*y)^-1, x^-2,y^-3, '_old_pow' in vars()

(x, x^2, x^4, x^2, y^-1*x^2, x, y^-3, True)

In [252]:
var('t')


t

In [253]:
# We need to construct a reduced pair, in the case of an infinite cyclic factor
# if v is a generator of \Z and (g,X), with elements in <v>, is the original pair, 
#    then we write a procedure to reduce it to (g,X'),
#    where X' is X, with all the nonzero powers of v removed 
#      EXCEPT ones with smallest positive, and largest negative, powers, if applicable.
# Eg1. X={1,v,v^2,v^-1,v^-4, v^17} -> X'= {1,v,v^-1}
# Eg2. X={v^2,v^5,v^7,v^-3,v^-100} -> X' = {v^2,v^-3}
# Eg3. X={1,v^3,v^5,v^20} -> X'={1,v^3}
# If g= v^i, i>0, and v^j in X' for some 0<j<i, then F_{g,X'}(t)=0

def series_must_be_0(pair, v, cr):
    # checks whether or not a pair satisfies the series=0 condition described above
    if cr>0:
        return False
    g, X = pair
    ep=0
    for _,ep in g.syllables():
        break
    if ep<0:
        v= v^-1
        ep=-ep
    return X.intersection(Set(v.powers(ep))-Set([G.one()]))!=Set()
    

def strip_redundant_powers(X, v, cr):
    '''
    v: either x or y, depending in which case, subset_of_cyclic_factor(X,v) would return True
    X: a subset of <v>
    cr: the characteristic of the factor generated by <v>, a number in 0,2,3,4,5,...
    returns X' as described above.
    '''
    if cr>0:
        # finite group, can't remove elements
        return X
    Xred = set()
    pos,neg = oo,-oo
    for a in X:
        ep=0
        for _,ep in a.syllables():
            break
        if ep>0:
            pos = min(ep,pos)
        if ep<0:
            neg = max(ep,neg)
    
    if G.one() in X:
        Xred.add(G.one())
    if pos in ZZ:
        Xred.add(v^pos)
    if neg in ZZ:
        Xred.add(v^neg)
    return Set(Xred)

#strip_redundant_powers(Set([y^2,y^5]), y, 0)
#series_must_be_0((x^2,Set([G.one(),x^-1,x^-2,x])),x,0)

In [254]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
        assume X is reduced in the inifnte cyclic case
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset) # also reduced since X is
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    elif subset_of_cyclic_factor(X,x):
        if g!=one:
            eqn = t*ZZ(g in Sx)
            for s in Sx-X:
                [gs] = s^-1*Set([g]) # use set so auto reduction is implemented
                Xs = s^-1*X
                Xs = strip_redundant_powers(Xs, x, m)
                if not series_must_be_0((gs,Xs),x,m):
                    if add_series_var((gs,Xs)) and ret_new_pairs:
                        newpairs.add((gs,Xs))
                    eqn += t*var_space[(gs,Xs)]
        else:
            eqn = 1*t^0
            for s in Sy:
                set_sinv = Set([s^-1]) # already reduced
                if add_series_var((s^-1,set_sinv)) and ret_new_pairs:
                    newpairs.add((s^-1,set_sinv))
                eqn += t*var_space[(s^-1,set_sinv)]
            for s in Sx-X:
                [gs] = s^-1*Set([g]) # use set so auto reduction is implemented
                Xs = s^-1*X
                Xs = strip_redundant_powers(Xs, x, m)
                if not series_must_be_0((gs,Xs),x,m):
                    if add_series_var((gs,Xs)) and ret_new_pairs:
                        newpairs.add((gs,Xs))
                    eqn += t*var_space[(gs,Xs)]
    else:
        # the y factor case
        if g!=one:
            eqn = t*ZZ(g in Sy)
            for s in Sy-X:
                [gs] = s^-1*Set([g]) # use set so auto reduction is implemented
                Xs = s^-1*X
                Xs = strip_redundant_powers(Xs, y, n)
                if not series_must_be_0((gs,Xs),y,n):
                    if add_series_var((gs,Xs)) and ret_new_pairs:
                        newpairs.add((gs,Xs))
                    eqn += t*var_space[(gs,Xs)]
        else:
            eqn = 1*t^0
            for s in Sx:
                set_sinv = Set([s^-1]) # already reduced
                if add_series_var((s^-1,set_sinv)) and ret_new_pairs:
                    newpairs.add((s^-1,set_sinv))
                eqn += t*var_space[(s^-1,set_sinv)]
            for s in Sy-X:
                [gs] = s^-1*Set([g]) # use set so auto reduction is implemented
                Xs = s^-1*X
                Xs = strip_redundant_powers(Xs, y, n)
                if not series_must_be_0((gs,Xs),y,n):
                    if add_series_var((gs,Xs)) and ret_new_pairs:
                        newpairs.add((gs,Xs))
                    eqn += t*var_space[(gs,Xs)]
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn
    

In [255]:
# Let us build the system of equations
from time import time

system = []
queue = [goal_pair]
st_time = time()
# use BFS
while queue:
    pair = queue.pop(0)
    #pretty_print(pair)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)
    end_time =  time()
    if end_time-st_time>10:
        break


In [260]:
# Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\emptyset}(t)} = -\frac{1}{{F_{1,\left\{1\right\}}(t)} - 2} \\
{F_{1,\left\{1\right\}}(t)} = t {F_{y^{-1},\left\{y^{-1}\right\}}(t)} + t {F_{y,\left\{y\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} + 1 \\
{F_{x^{2},\left\{x^{2}\right\}}(t)} = {F_{1,\left\{x^{2}\right\}}(t)} {F_{x^{2},\left\{1, x^{2}\right\}}(t)} \\
{F_{y,\left\{y\right\}}(t)} = {F_{1,\left\{y\right\}}(t)} {F_{y,\left\{y, 1\right\}}(t)} \\
{F_{y^{-1},\left\{y^{-1}\right\}}(t)} = {F_{y^{-1},\left\{1, y^{-1}\right\}}(t)} {F_{1,\left\{y^{-1}\right\}}(t)} \\
{F_{x^{2},\left\{1, x^{2}\right\}}(t)} = t {F_{x,\left\{x^{2}, x\right\}}(t)} \\
{F_{1,\left\{x^{2}\right\}}(t)} = -\frac{1}{{F_{1,\left\{1, x^{2}\right\}}(t)} - 2} \\
{F_{y,\left\{y, 1\right\}}(t)} = t \\
{F_{1,\left\{y\right\}}(t)} = -\frac{1}{{F_{1,\left\{y, 1\right\}}(t)} - 2} \\
{F_{y^{-1},\left\{1, y^{-1}\right\}}(t)} = t \\
{F_{1,\left\{y^{-1}\right\}}(t)} = -\frac{1}{{F_{1,\left\{1, y^{-1}\right\}}(t)} - 2} \\
{F_{x,\left\{x^{2}, x\right\}}(t)} = {F

v0 == -1/(v1 - 2)

v1 == t*v2 + t*v3 + t*v4 + 1

v4 == v5*v6

v3 == v7*v8

...

v15 == -1/(v18 - 2)

v16 == t

v17 == v15*v19

v18 == t*v2 + t*v3 + 1

v19 == 0

In [262]:
# Finally solve the system



# save so we dont lose it
orig_system = list(system)

In [263]:
# first, isolate variables on the LHS

for i, eqn in enumerate(system):
    system[i] = solve(eqn,eqn.lhs())[0]
    

In [264]:
for eqn in system:
    pretty_print(eqn)
print(len(system))

v0 == -1/(v1 - 2)

v1 == t*v2 + t*v3 + t*v4 + 1

v4 == v5*v6

v3 == v7*v8

v2 == v10*v9

...

v16 == t

v17 == v15*v19

v18 == t*v2 + t*v3 + 1

v19 == 0

20


In [265]:
# now make some substitutions

change = 1
numit = 0

while change:
    change = 0
    # build a dependency graph
    eq_by_lhs = {eq.lhs():eq for eq in system}
    dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}
    vars_det = [v for v,args in dep_gr.items() if not args] # the completely solved vars

    rev_dep_gr = {v:set() for v in eq_by_lhs} # diagraph by reversing the arcs of dep_gr
    for v,ag in dep_gr.items():
        for w in ag:
            rev_dep_gr[w].add(v)

    # apply BFS to make the substitutions possible
    vis = set(vars_det)
    queue = list(vis)
    done = set()
    while queue:
        nxtvar = queue.pop(0)
        # check if all RHS variables are visited before
        sub_dict = {}
        for dps in dep_gr[nxtvar]:
            if dps not in done:
                continue # can't substitute, a RHS varible is not yet reduced
            sub_dict[dps] = eq_by_lhs[dps].rhs()
            change = 1
        eq_by_lhs[nxtvar] = eq_by_lhs[nxtvar].subs(sub_dict)
        eq_by_lhs[nxtvar].expand()
        for pt in rev_dep_gr[nxtvar]:
            if pt not in vis:
                queue.append(pt)
                vis.add(pt)
        done.add(nxtvar)
    
    for i, eq in enumerate(system):
        system[i] = eq_by_lhs[eq.lhs()]
    numit+=1

print('Number of iteration taken to substitute',numit)
        


Number of iteration taken to substitute 2


In [266]:
for eq in eq_by_lhs.values():
    pretty_print(eq)

v0 == -1/(t^2*v7 + t^2*v9 + t*v4 - 1)

v1 == t^2*v7 + t^2*v9 + t*v4 + 1

v4 == -t^2*v15/(t^2*v7 + t^2*v9 - 1)

v3 == t*v7

v2 == t*v9

...

v15 == -1/(t^2*v7 + t^2*v9 - 1)

v16 == t

v17 == 0

v18 == t^2*v7 + t^2*v9 + 1

v19 == 0

In [267]:
# get rid of every irrelevant variable
start = var_space[goal_pair]
eq_by_lhs = {eq.lhs():eq for eq in system}
dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}

comp_set = {start}
comp = [start]
ind = 0
while ind<len(comp):
    for v in dep_gr[comp[ind]]:
        if v not in comp_set:
            comp.append(v)
            comp_set.add(v)
    ind += 1

system = [eq for eq in system if eq.lhs() in comp_set]

In [268]:
pretty_print(comp)
for eq in system:
    print(eq)
    pretty_print(eq)

[v0, v4, v9, v7, v15]

v0 == -1/(t^2*v7 + t^2*v9 + t*v4 - 1)


v0 == -1/(t^2*v7 + t^2*v9 + t*v4 - 1)

v4 == -t^2*v15/(t^2*v7 + t^2*v9 - 1)


v4 == -t^2*v15/(t^2*v7 + t^2*v9 - 1)

...

v7 == -1/(t^2*v7 + t*v4 - 1)

v9 == -1/(t^2*v9 + t*v4 - 1)


v9 == -1/(t^2*v9 + t*v4 - 1)

v15 == -1/(t^2*v7 + t^2*v9 - 1)


v15 == -1/(t^2*v7 + t^2*v9 - 1)

In [269]:
# make every equation eplicit. delete loops
# maybe not a good idea due to the posssible interference of radicals

# for i, eqn in enumerate(system):
#     system[i] = solve(eqn,eqn.lhs())[0]

In [270]:
for eq in system:
    print(latex(eq)+r'\\')
    #pretty_print(eq)

{F_{1,\emptyset}(t)} = -\frac{1}{t^{2} {F_{1,\left\{y\right\}}(t)} + t^{2} {F_{1,\left\{y^{-1}\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} - 1} \\
{F_{x^{2},\left\{x^{2}\right\}}(t)} = -\frac{t^{2} {F_{1,\left\{x^{2}, x\right\}}(t)}}{t^{2} {F_{1,\left\{y\right\}}(t)} + t^{2} {F_{1,\left\{y^{-1}\right\}}(t)} - 1} \\
{F_{1,\left\{y\right\}}(t)} = -\frac{1}{t^{2} {F_{1,\left\{y\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} - 1} \\
{F_{1,\left\{y^{-1}\right\}}(t)} = -\frac{1}{t^{2} {F_{1,\left\{y^{-1}\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} - 1} \\
{F_{1,\left\{x^{2}, x\right\}}(t)} = -\frac{1}{t^{2} {F_{1,\left\{y\right\}}(t)} + t^{2} {F_{1,\left\{y^{-1}\right\}}(t)} - 1} \\


In [271]:
# order the component based on the shortest distance from the start in decreasing order
# BFS already put them in increasing order
comp.reverse()
comp.pop(); # get rid of start; the only variable we don't want to eliminate
comp

[v15, v7, v9, v4]

In [272]:
def eliminate(system,xs, show_steps = False, fully_simplify=False, quit_at_zero= False):
    if show_steps:
        sols = []
        for x in xs:
            sols.append((x, eliminate(system,[x], fully_simplify=fully_simplify)))
            system = sols[-1][1]
            if quit_at_zero and system == [0]*len(system):
                return sols
        return sols
    sols = maxima.eliminate(system,xs)
    func = lambda ex: ex.simplify_full() if fully_simplify else ex
    return [func(expr.sage()) for expr in sols]

In [273]:
# Expression involving t and F(t) to set = 0

impl_solset = []
steps = eliminate(system, comp, True, quit_at_zero=True)
while steps and 0 in steps[-1][-1]:
    steps.pop()
if steps:
    impl_solset = steps[-1][-1]

'''
possible_impl_solns = []
for comp_perm in Permutations(comp):
    comp_perm = list(comp_perm)
    print(comp_perm,type(comp_perm))
    possible_impl_solns_perm = eliminate(system, comp_perm)
    possible_impl_solns.extend(possible_impl_solns_perm)
    for impl_soln in possible_impl_solns_perm:
        pretty_print(impl_soln)
        if impl_soln!=0:
            break # found an actual solution
    else:
        continue
    break
''';

In [274]:

print(len(impl_solset))
for sol in impl_solset:
    pretty_print(sol==0)
    print(sol.args())

for sol in impl_solset:
    print(latex(sol)+r"=0\\")
    
#solve(system, comp, algorithm='sympy')
#impl_soln(v0=sqrt(1/(1-4*t^2))).simplify_full()

1


(366912*t^18*v0^14 - 7168*(21*v0^14 - 13*v0^13)*t^17 - 2048*(285*v0^14 + 344*v0^13 - 77*v0^12)*t^16 + 16*(16496*v0^14 + 16720*v0^13 - 4256*v0^12 + 9221*v0^11)*t^15 + 256*(1158*v0^14 + 2944*v0^13 + 900*v0^12 - 1323*v0^11 + 171*v0^10)*t^14 - 512*(240*v0^14 + 560*v0^13 + 200*v0^12 + 229*v0^11 + 424*v0^10 - 93*v0^9)*t^13 - (80256*v0^14 + 324608*v0^13 + 296832*v0^12 - 223168*v0^11 - 260928*v0^10 + 47744*v0^9 - 20709*v0^8)*t^12 + 16*(1536*v0^14 + 5632*v0^13 + 5184*v0^12 + 4184*v0^11 + 10752*v0^10 + 3600*v0^9 - 5363*v0^8 + 411*v0^7)*t^11 + 32*(480*v0^14 + 2560*v0^13 + 3968*v0^12 - 960*v0^11 - 6144*v0^10 - 2080*v0^9 + 707*v0^8 - 664*v0^7 + 141*v0^6)*t^10 - 4*(448*v0^14 + 2240*v0^13 + 3264*v0^12 + 5208*v0^11 + 17600*v0^10 + 16632*v0^9 - 11879*v0^8 - 13685*v0^7 + 2186*v0^6 - 282*v0^5)*t^9 - 8*(192*v0^14 + 1280*v0^13 + 2752*v0^12 + 256*v0^11 - 6272*v0^10 - 5920*v0^9 + 157*v0^8 + 64*v0^7 - 450*v0^6 + 844*v0^5 - 40*v0^4)*t^8 + 64*(60*v0^11 + 320*v0^10 + 496*v0^9 - 30*v0^8 - 558*v0^7 - 185*v0^6 + 103*v0^5 - 10*v0^4 + 2*v0^3)*t^7 + v0^8 + 2*(32*v0^14 + 256*v0^13 + 704*v0^12 + 288*v0^11 - 2112*v0^10 - 3424*v0^9 - 979*v0^8 - 168*v0^7 - 847*v0^6 + 1740*v0^5 + 1772*v0^4 - 240*v0^3 + 8*v0^2)*t^6 + 8*v0^7 - 16*(24*v0^11 + 160*v0^10 + 344*v0^9 + 104*v0^8 - 520*v0^7 - 497*v0^6 + 71*v0^5 + 144*v0^4 + 8*v0^2)*t^5 + 22*v0^6 + 16*(15*v0^8 + 80*v0^7 + 124*v0^6 - 122*v0^4 - 40*v0^3 + 24*v0^2)*t^4 + 16*v0^5 + 4*(4*v0^11 + 32*v0^10 + 88*v0^9 + 57*v0^8 - 159*v0^7 - 275*v0^6 - 35*v0^5 + 170*v0^4 + 94*v0^3 + 16*v0^2 + 8*v0)*t^3 - 31*v0^4 - 8*(3*v0^8 + 20*v0^7 + 43*v0^6 + 16*v0^5 - 54*v0^4 - 52*v0^3 + 8*v0^2 + 16*v0)*t^2 - 56*v0^3 - 8*v0^2 + 32*v0 + 16)^8*t^864 == 0

(t, v0)
{\left(366912 \, t^{18} {F_{1,\emptyset}(t)}^{14} - 7168 \, {\left(21 \, {F_{1,\emptyset}(t)}^{14} - 13 \, {F_{1,\emptyset}(t)}^{13}\right)} t^{17} - 2048 \, {\left(285 \, {F_{1,\emptyset}(t)}^{14} + 344 \, {F_{1,\emptyset}(t)}^{13} - 77 \, {F_{1,\emptyset}(t)}^{12}\right)} t^{16} + 16 \, {\left(16496 \, {F_{1,\emptyset}(t)}^{14} + 16720 \, {F_{1,\emptyset}(t)}^{13} - 4256 \, {F_{1,\emptyset}(t)}^{12} + 9221 \, {F_{1,\emptyset}(t)}^{11}\right)} t^{15} + 256 \, {\left(1158 \, {F_{1,\emptyset}(t)}^{14} + 2944 \, {F_{1,\emptyset}(t)}^{13} + 900 \, {F_{1,\emptyset}(t)}^{12} - 1323 \, {F_{1,\emptyset}(t)}^{11} + 171 \, {F_{1,\emptyset}(t)}^{10}\right)} t^{14} - 512 \, {\left(240 \, {F_{1,\emptyset}(t)}^{14} + 560 \, {F_{1,\emptyset}(t)}^{13} + 200 \, {F_{1,\emptyset}(t)}^{12} + 229 \, {F_{1,\emptyset}(t)}^{11} + 424 \, {F_{1,\emptyset}(t)}^{10} - 93 \, {F_{1,\emptyset}(t)}^{9}\right)} t^{13} - {\left(80256 \, {F_{1,\emptyset}(t)}^{14} + 324608 \, {F_{1,\emptyset}(t)}^{13} + 296832 \

In [275]:
# the remaning variable, including the one we wish to solve for
var_rem = comp[-len(impl_solset)+1:]+[start]
print(var_rem)
print(list(map(latex,var_rem)))

[v15, v7, v9, v4, v0]
[{F_{1,\left\{x^{2}, x\right\}}(t)}, {F_{1,\left\{y\right\}}(t)}, {F_{1,\left\{y^{-1}\right\}}(t)}, {F_{x^{2},\left\{x^{2}\right\}}(t)}, {F_{1,\emptyset}(t)}]


In [276]:
have_expl = 1
try:
    expl_slns = solve(impl_solset, var_rem, solution_dict=True) # can we get explicit solutions?
    print(expl_slns)
except: 
    # can't get explicit solutions. try for implicit ones. perhaps reduced a bit?
    impl_solset = solve(impl_solset, var_rem)
    have_expl = 0
    for sol in impl_solset:
        pretty_print(sol)
    

[{v15: r93, v7: r94, v9: r95, v4: r96, v0: -1/6*(24*t^5 - 96*t^4 + 6*t^3 + 48*t^2 - (63*t^6 + 24*t^5 - 48*t^4 + 2*t^3 + 12*t^2 - 1)*sqrt(-sqrt(3)*(sqrt(3)*(64*t^8 + 1792*t^7 + 224*t^6 + 896*t^5 - 32*t^4 + 48*t^3 - 48*t^2 + (3969*t^12 + 3024*t^11 - 5472*t^10 - 2052*t^9 + 3912*t^8 + 384*t^7 - 1274*t^6 + 240*t^4 - 4*t^3 - 24*t^2 + 1)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(2/3) + 4*(912*t^10 + 1656*t^9 - 1656*t^8 - 352*t^7 + 1165*t^6 - 8*t^5 - 352*t^4 - 2*t^3 + 52*t^2 - 3)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 -

In [239]:
if have_expl:
    # if the solutions are explicit, then test them out too see if we have the desired one
    oldt = t
    R.<t>= PolynomialRing(ZZ,1,order='negdegrevlex')
    tt= t
    t = oldt
    for sol in expl_slns:
        F = sol[start]
        pretty_print(F)
        L = limit(F,t=0)
        pretty_print(LatexExpr(r'\lim_{t\to 0}%s=%s'%(latex(start),L)))
        if L!=1:
            print('limit at t=0 is different from 1, discard!')
            #print(F.taylor(t,0,20))
        else:
            print('possible desired series solutions!')
            pretty_print(F)
            prec = 16
            pretty_print('Series Expansion around %s of degree up to %d: '%(LatexExpr('t=0'),prec))
            P = F.taylor(t, 0, prec)
            P = R(P(tt))
            pretty_print(P)
            print(latex(F)+" = "+latex(P)+r"+ O(t^{%d})"%(prec+1))

            

-1/6*(24*t^5 - 96*t^4 + 6*t^3 + 48*t^2 - (63*t^6 + 24*t^5 - 48*t^4 + 2*t^3 + 12*t^2 - 1)*sqrt(-sqrt(3)*(sqrt(3)*(64*t^8 + 1792*t^7 + 224*t^6 + 896*t^5 - 32*t^4 + 48*t^3 - 48*t^2 + (3969*t^12 + 3024*t^11 - 5472*t^10 - 2052*t^9 + 3912*t^8 + 384*t^7 - 1274*t^6 + 240*t^4 - 4*t^3 - 24*t^2 + 1)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(2/3) + 4*(912*t^10 + 1656*t^9 - 1656*t^8 - 352*t^7 + 1165*t^6 - 8*t^5 - 352*t^4 - 2*t^3 + 52*t^2 - 3)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(1/3) + 9)*sqrt((64*t^8 + 1792*t^7 + 224*t^6 + 896*t^5 - 32*t^4 + 48*t^3 - 48*t^2 + (3969*t^12 + 3024*t^11 - 5472*t^10 - 2052*t^9 + 3912*t^8 + 384*t^7 - 1274*t^6 + 240*t^4 - 4*t^3 - 24*t^2 + 1)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(2/3) - 2*(912*t^10 + 1656*t^9 - 1656*t^8 - 352*t^7 + 1165*t^6 - 8*t^5 - 352*t^4 - 2*t^3 + 52*t^2 - 3)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(1/3) + 9)/((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(1/3)) + 36*(4162*t^15 - 16200*t^14 - 9168*t^13 + 21762*t^12 + 12988*t^11 - 20768*t^10 - 5907*t^9 + 9104*t^8 + 880*t^7 - 2215*t^6 + 24*t^5 + 336*t^4 - 11*t^3 - 28*t^2 + 1)*((13096*t^12 - 62976*t^11 - 14976*t^10 - 66496*t^9 - 10368*t^8 - 9984*t^7 - 3592*t^6 + 3456*t^5 - 144*t^4 + 1080*t^3 + 216*t^2 + 36*sqrt(91/3*t^10 - 800/3*t^9 + 464/3*t^8 + 152*t^7 - 244/3*t^6 - 64*t^5 + 31/3*t^4 + 32/3*t^3 - 2/3*t)*(66*t^7 - 32*t^6 + 112*t^5 - 8*t^4 + 60*t^3 + 9*t) - 27)/(250047*t^18 + 285768*t^17 - 462672*t^16 - 397818*t^15 + 513540*t^14 + 241920*t^13 - 332559*t^12 - 68832*t^11 + 129456*t^10 + 9620*t^9 - 32328*t^8 - 288*t^7 + 5361*t^6 - 72*t^5 - 576*t^4 + 6*t^3 + 36*t^2 - 1))^(1/3))/((3969*t^12 + 3024*t^11 - 5472*t^10 - 2052*t^9 + 3912*t^8 + 384*t^7 - 1274*t^6 + 240*t^4 - 4*t^3 - 24*t^2 + 1)*sqrt((64*

RuntimeError: ECL says: Console interrupt.

In [277]:
# Take the reduced implicit system of equations, with the list of variables to eliminate,
#  and dump them to a maple data file
import os
print(os.getcwd())
fdir = 'maple_data_files'
if not os.path.isdir(fdir):
    os.mkdir(fdir)
fname = 'Z%dZ%d-dat.maple'%(m,n)
fname = os.path.join(fdir,fname)
if not os.path.exists(fname):
    fd = open(fname, 'w')
    fd.write('syst:='+str(impl_solset).replace('==','=')+';')
    fd.write('\n')
    fd.write('varlist:='+str((sorted(var_rem, key = lambda vv: int(str(vv)[1:]))))+';')
    fd.write('\n')
    fd.close()

/home/sage/OneDrive - sfu.ca/Research/Msc_math_sfu/Marni/cogrowth_computations


# Solving a system iteratively

Suppose we are given a system of the form
$$\begin{align*}
\Phi_1 (v_1,...v_q) &= 0\\
\Phi_2 (v_1,...v_q) &= 0\\
\vdots\\
\Phi_r (v_1,...v_q) &= 0
\end{align*}$$ where $q\ge r$. We want to solve for $v_1,...,v_r$ in terms of the remaining $q-r$ variables.
We can do this approximately using a fixed point iterative method, assumming convergence.

First, rewrite the $i$-th equation above as $$\Phi_i (v_1,...,v_q)+v_i = v_i.$$
Generate iterates $v_i^{(k)}$ with $v_i^{(0)}$ determined, with the update formula as $$\Phi_i (v_1^{(k)},...,v_q^{(k)})+v_i^{(k)} = v_i^{(k+1)}.$$ For large $k$, the iterates make an approximate solution.

In the case where $r=q-1$ and $v_q=t$, and the system is algebraic, we can expect approximations to the power series solution of $v_i$ to be increasingly accurate. If we take $v_i^{(0)}=0$ for each $i$, then we should get increasing higher oder terms each iteration, with $v_i^{(k)}$ converging to the series as $k\to \infty$.

In [ ]:
def homogenize(eqn):
    '''
    eqn: an equation or expression (to set = 0)
    returns an expression, to set = 0, that is equivalent to applying eqn
    '''
    if eqn.is_relational():
        return eqn.rhs()-eqn.lhs()
    return eqn


def iter_solve(system, vrs, vinit=None, maxiter=5, fully_simplify=False, taylor_poly=True):
    if len(system)<len(vrs):
        raise Exception('number of variables must be at least the number of equations')
    if not system:
        return [] # no system to solve
    if vinit is None:
        vinit = {v:0*t for v in vrs}
    system = list(map(homogenize, system))
    it_sols = [vinit]
    k = 1
    while k<=maxiter:
        vrs_k = {}
        for vi,eqi in zip(vrs,system):
            vrs_k[vi] = it_sols[-1][vi]+eqi.subs(it_sols[-1])
            if fully_simplify:
                vrs_k[vi] = vrs_k[vi].simplify_full()
            if taylor_poly:
                vrs_k[vi] = vrs_k[vi].taylor(t,0,k)
        it_sols.append(vrs_k)
        k+=1
    return it_sols
    

In [ ]:
for ans in iter_solve(orig_system,list(var_space.values()),fully_simplify=False,maxiter=20,taylor_poly=True):
    pretty_print(ans[start])

In [ ]:
# STOP HERE

In [ ]:
print(ss,file=open('ss.txt','w'))

In [ ]:
eg = [-((t^3*v0^2 - v0^2 - 2*v0 - 1)*v13^3 + (3*v0^2 + 4*v0 + 1)*v13^2 + v0^2 - (3*v0^2 + 2*v0)*v13)*t^24*v13^6, (t^6*v13^5 - t^6*v13^4 - 2*t^3*v13^3 + 2*t^3*v13^2 - (t^3 - 1)*v13 - 1)*t^18*v13^4]
maxima.eliminate(eg, [v13])

In [ ]:
for v,ss in eliminate(system, [v3,v15,v2,v4,v13], True):
    print('+'*100)
    pretty_print(v,':')
    for s in ss:
        print('|'*50)
        print(bool(s==0))
        pretty_print(s)
    print (ss)
    if len(ss)==1:
        pretty_print(ss[0])

In [ ]:
solve(impl_solset,[v0,v13])

In [ ]:
type(expl_slns[0][v0])

In [ ]:
F = -1/(4*t^2+2*t+1)
((8*t^3-1)*F^3-F^2+F+1).simplify_full()

In [ ]:
var('F')
ans=(solve(((8*t^3-1)*F^3-F^2+F+1), F))[-1]

In [238]:
len(expl_slns)

11

In [ ]:

    
for soldict in solve(orig_system, [v for v in var_space.values()],solution_dict=True):
    pretty_print(soldict[v0])
    for eqn in orig_system:
        print(bool(eqn.subs(soldict)))
    print('-'*20)
    print(bool(soldict[v0]==ans.rhs()))
    


In [278]:
system = orig_system

In [282]:
print(str(impl_solset).replace('==','='))
print(sorted(var_space.values(), key = lambda vv: int(str(vv)[1:])))

[(366912*t^18*v0^14 - 7168*(21*v0^14 - 13*v0^13)*t^17 - 2048*(285*v0^14 + 344*v0^13 - 77*v0^12)*t^16 + 16*(16496*v0^14 + 16720*v0^13 - 4256*v0^12 + 9221*v0^11)*t^15 + 256*(1158*v0^14 + 2944*v0^13 + 900*v0^12 - 1323*v0^11 + 171*v0^10)*t^14 - 512*(240*v0^14 + 560*v0^13 + 200*v0^12 + 229*v0^11 + 424*v0^10 - 93*v0^9)*t^13 - (80256*v0^14 + 324608*v0^13 + 296832*v0^12 - 223168*v0^11 - 260928*v0^10 + 47744*v0^9 - 20709*v0^8)*t^12 + 16*(1536*v0^14 + 5632*v0^13 + 5184*v0^12 + 4184*v0^11 + 10752*v0^10 + 3600*v0^9 - 5363*v0^8 + 411*v0^7)*t^11 + 32*(480*v0^14 + 2560*v0^13 + 3968*v0^12 - 960*v0^11 - 6144*v0^10 - 2080*v0^9 + 707*v0^8 - 664*v0^7 + 141*v0^6)*t^10 - 4*(448*v0^14 + 2240*v0^13 + 3264*v0^12 + 5208*v0^11 + 17600*v0^10 + 16632*v0^9 - 11879*v0^8 - 13685*v0^7 + 2186*v0^6 - 282*v0^5)*t^9 - 8*(192*v0^14 + 1280*v0^13 + 2752*v0^12 + 256*v0^11 - 6272*v0^10 - 5920*v0^9 + 157*v0^8 + 64*v0^7 - 450*v0^6 + 844*v0^5 - 40*v0^4)*t^8 + 64*(60*v0^11 + 320*v0^10 + 496*v0^9 - 30*v0^8 - 558*v0^7 - 185*v0^6 + 1

In [280]:
str(system).replace('==','=')


'[v0 = -1/(v1 - 2), v1 = t*v2 + t*v3 + t*v4 + 1, v4 = v5*v6, v3 = v7*v8, v2 = v10*v9, v6 = t*v11, v5 = -1/(v12 - 2), v8 = t, v7 = -1/(v13 - 2), v10 = t, v9 = -1/(v14 - 2), v11 = v15*v16, v12 = t*v17 + t*v2 + t*v3 + 1, v13 = t*v3 + t*v4 + 1, v14 = t*v2 + t*v4 + 1, v15 = -1/(v18 - 2), v16 = t, v17 = v15*v19, v18 = t*v2 + t*v3 + 1, v19 = 0]'